## Preprocessing

### 데이터 불러오기

In [50]:
# 사용 라이브러리

import pandas as pd

In [51]:
# 데이터 불러오기

train_data = pd.read_table('./data/ratings_train.txt')
test_data = pd.read_table('./data/ratings_test.txt')


In [52]:
train_data.info() # 150000 row, 5 null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [53]:
test_data.info() # 50000 row, 3 null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


### 데이터 정제

#### 결측치 처리

In [54]:
# 결측치 확인

train_data[train_data['document'].isnull()==True]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [55]:
# 결측치 확인

test_data[test_data['document'].isnull()==True]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [56]:
# 결측치 제거

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [57]:
# 결측치 확인

print(train_data['document'].isnull().sum(), test_data['document'].isnull().sum())

0 0


#### 중복값 처리

In [58]:
# train_data 중복값

print('id 중복값: ', train_data['id'].notnull().sum() - len(train_data['id'].unique()))
print('document 중복값: ', train_data['document'].notnull().sum() - len(train_data['document'].unique()))

print('label 값: ', train_data['label'].unique()) # 0, 1

id 중복값:  0
document 중복값:  3813
label 값:  [0 1]


In [59]:
# test_data 중복값

print('id 중복값: ', test_data['id'].notnull().sum() - len(test_data['id'].unique()))
print('document 중복값: ', test_data['document'].notnull().sum() - len(test_data['document'].unique()))

print('label 값: ', test_data['label'].unique()) # 0, 1

id 중복값:  0
document 중복값:  840
label 값:  [1 0]
